In [ ]:
# Pascal VOC Dataset - Exploratory Data Analysis (EDA)

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import xml.etree.ElementTree as ET
from collections import Counter, defaultdict
import seaborn as sns

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
## 1. Dataset Overview and Setup

# Pascal VOC 2012 dataset paths
VOC_ROOT = './data/VOCdevkit/VOC2012'
IMAGES_DIR = os.path.join(VOC_ROOT, 'JPEGImages')
ANNOTATIONS_DIR = os.path.join(VOC_ROOT, 'Annotations')
SEGMENTATION_CLASS_DIR = os.path.join(VOC_ROOT, 'SegmentationClass')
SEGMENTATION_OBJECT_DIR = os.path.join(VOC_ROOT, 'SegmentationObject')
IMAGESETS_DIR = os.path.join(VOC_ROOT, 'ImageSets', 'Segmentation')

# Pascal VOC class names (20 object classes + background)
VOC_CLASSES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

# Color palette for visualization (RGB)
VOC_COLORMAP = [
    [0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
    [128, 0, 128], [0, 128, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0],
    [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128], [64, 128, 128],
    [192, 128, 128], [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
    [0, 64, 128]
]

print("Pascal VOC 2012 Dataset Paths:")
print(f"Root: {VOC_ROOT}")
print(f"Images: {IMAGES_DIR}")
print(f"Annotations: {ANNOTATIONS_DIR}")
print(f"Segmentation Masks: {SEGMENTATION_CLASS_DIR}")
print(f"\nNumber of classes: {len(VOC_CLASSES)}")


In [ ]:
## 2. Load Dataset Splits

def load_image_ids(split='train'):
    """Load image IDs for a given split (train/val/trainval)"""
    split_file = os.path.join(IMAGESETS_DIR, f'{split}.txt')
    if os.path.exists(split_file):
        with open(split_file, 'r') as f:
            return [line.strip() for line in f.readlines()]
    return []

# Load all splits
train_ids = load_image_ids('train')
val_ids = load_image_ids('val')
trainval_ids = load_image_ids('trainval')

print("Dataset Split Information:")
print(f"Training set: {len(train_ids)} images")
print(f"Validation set: {len(val_ids)} images")
print(f"Train+Val set: {len(trainval_ids)} images")
print(f"\nExample image IDs: {train_ids[:5]}")


## 3. Analyze Image Dimensions

def get_image_dimensions(image_ids, max_samples=None):
    """Get dimensions of images in the dataset"""
    dimensions = []
    if max_samples:
        image_ids = image_ids[:max_samples]
    
    for img_id in image_ids:
        img_path = os.path.join(IMAGES_DIR, f'{img_id}.jpg')
        if os.path.exists(img_path):
            img = Image.open(img_path)
            dimensions.append(img.size)  # (width, height)
    return dimensions

# Analyze dimensions for training set (sample for speed)
print("Analyzing image dimensions...")
dimensions = get_image_dimensions(train_ids, max_samples=500)

widths = [d[0] for d in dimensions]
heights = [d[1] for d in dimensions]
aspect_ratios = [w/h for w, h in dimensions]

print(f"\nImage Dimensions Statistics (based on {len(dimensions)} samples):")
print(f"Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.1f}")
print(f"Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.1f}")
print(f"Aspect Ratio - Min: {min(aspect_ratios):.2f}, Max: {max(aspect_ratios):.2f}, Mean: {np.mean(aspect_ratios):.2f}")

# Visualize dimensions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(widths, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Width (pixels)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Image Widths')

axes[1].hist(heights, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Height (pixels)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Image Heights')

axes[2].hist(aspect_ratios, bins=30, edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Aspect Ratio (W/H)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Distribution of Aspect Ratios')

plt.tight_layout()
plt.show()

In [ ]:
## 4. Analyze Segmentation Masks - Class Distribution

def analyze_segmentation_mask(mask_path):
    """Analyze a single segmentation mask and return class counts"""
    mask = Image.open(mask_path)
    mask_array = np.array(mask)
    
    # Get unique classes (excluding 255 which is the boundary/ignore class)
    unique_classes = np.unique(mask_array)
    unique_classes = unique_classes[unique_classes != 255]
    
    # Count pixels per class
    class_pixels = {}
    for cls in unique_classes:
        class_pixels[cls] = np.sum(mask_array == cls)
    
    return unique_classes, class_pixels

# Analyze class distribution across training set (sample for speed)
print("Analyzing segmentation masks...")
class_counts = Counter()
image_class_presence = defaultdict(int)  # How many images contain each class
total_masks_analyzed = 0

for img_id in train_ids[:500]:  # Sample 500 images
    mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
    if os.path.exists(mask_path):
        unique_classes, class_pixels = analyze_segmentation_mask(mask_path)
        
        # Count total occurrences
        for cls in unique_classes:
            class_counts[cls] += 1
            
        total_masks_analyzed += 1

print(f"\nAnalyzed {total_masks_analyzed} segmentation masks")
print(f"Number of unique classes found: {len(class_counts)}")

# Create visualization
class_names = [VOC_CLASSES[cls] for cls in sorted(class_counts.keys())]
class_frequencies = [class_counts[cls] for cls in sorted(class_counts.keys())]

plt.figure(figsize=(14, 6))
bars = plt.bar(range(len(class_names)), class_frequencies, edgecolor='black', alpha=0.7)
plt.xlabel('Class')
plt.ylabel('Number of Images Containing Class')
plt.title(f'Class Distribution in Segmentation Masks (Sample of {total_masks_analyzed} images)')
plt.xticks(range(len(class_names)), class_names, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Color bars by class
for i, bar in enumerate(bars):
    cls_idx = sorted(class_counts.keys())[i]
    if cls_idx < len(VOC_COLORMAP):
        bar.set_color([c/255.0 for c in VOC_COLORMAP[cls_idx]])

plt.tight_layout()
plt.show()

# Print top 10 most common classes
print("\nTop 10 Most Common Classes:")
for i, (cls, count) in enumerate(class_counts.most_common(10), 1):
    print(f"{i}. {VOC_CLASSES[cls]}: {count} images")

## 5. Visualize Sample Images with Segmentation Masks

def visualize_sample(img_id, show_colorful=True):
    """Visualize an image alongside its segmentation mask"""
    img_path = os.path.join(IMAGES_DIR, f'{img_id}.jpg')
    mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
    
    if not os.path.exists(img_path) or not os.path.exists(mask_path):
        return None, None
    
    img = Image.open(img_path)
    mask = Image.open(mask_path)
    mask_array = np.array(mask)
    
    # Get unique classes in this mask
    unique_classes = np.unique(mask_array)
    unique_classes = unique_classes[unique_classes != 255]
    
    # Create colored mask
    if show_colorful:
        colored_mask = np.zeros((*mask_array.shape, 3), dtype=np.uint8)
        for cls in unique_classes:
            if cls < len(VOC_COLORMAP):
                colored_mask[mask_array == cls] = VOC_COLORMAP[cls]
    
    return img, (colored_mask if show_colorful else mask_array), unique_classes

# Visualize multiple samples
n_samples = 6
sample_ids = train_ids[:n_samples]

fig, axes = plt.subplots(n_samples, 3, figsize=(15, 4*n_samples))

for i, img_id in enumerate(sample_ids):
    img, mask, unique_classes = visualize_sample(img_id)
    
    if img is not None:
        # Original image
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f'Original Image: {img_id}')
        axes[i, 0].axis('off')
        
        # Segmentation mask (colored)
        axes[i, 1].imshow(mask)
        axes[i, 1].set_title('Segmentation Mask (Colored)')
        axes[i, 1].axis('off')
        
        # Overlay
        axes[i, 2].imshow(img)
        axes[i, 2].imshow(mask, alpha=0.5)
        axes[i, 2].set_title('Overlay')
        axes[i, 2].axis('off')
        
        # Print classes present
        class_names = [VOC_CLASSES[cls] for cls in unique_classes if cls < len(VOC_CLASSES)]
        print(f"{img_id}: {', '.join(class_names)}")

plt.tight_layout()
plt.show()

In [ ]:
## 6. Pixel-Level Class Distribution

def calculate_pixel_distribution(image_ids, max_samples=200):
    """Calculate the distribution of pixels across all classes"""
    pixel_counts = defaultdict(int)
    total_pixels = 0
    
    for img_id in image_ids[:max_samples]:
        mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
        if os.path.exists(mask_path):
            mask = np.array(Image.open(mask_path))
            
            # Count pixels for each class
            unique, counts = np.unique(mask, return_counts=True)
            for cls, count in zip(unique, counts):
                if cls != 255:  # Ignore boundary class
                    pixel_counts[cls] += count
                    total_pixels += count
    
    return pixel_counts, total_pixels

print("Calculating pixel-level class distribution...")
pixel_counts, total_pixels = calculate_pixel_distribution(train_ids, max_samples=300)

# Calculate percentages
class_percentages = {cls: (count / total_pixels) * 100 for cls, count in pixel_counts.items()}

# Sort by percentage
sorted_classes = sorted(class_percentages.items(), key=lambda x: x[1], reverse=True)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot
classes = [VOC_CLASSES[cls] for cls, _ in sorted_classes]
percentages = [pct for _, pct in sorted_classes]

bars = ax1.barh(range(len(classes)), percentages, edgecolor='black', alpha=0.7)
ax1.set_yticks(range(len(classes)))
ax1.set_yticklabels(classes)
ax1.set_xlabel('Percentage of Total Pixels (%)')
ax1.set_title('Pixel Distribution Across Classes')
ax1.invert_yaxis()

# Color bars
for i, (cls, _) in enumerate(sorted_classes):
    if cls < len(VOC_COLORMAP):
        bars[i].set_color([c/255.0 for c in VOC_COLORMAP[cls]])

# Pie chart for top classes
top_n = 10
top_classes = sorted_classes[:top_n]
other_pct = sum(pct for _, pct in sorted_classes[top_n:])

pie_labels = [VOC_CLASSES[cls] for cls, _ in top_classes]
pie_values = [pct for _, pct in top_classes]

if other_pct > 0:
    pie_labels.append('Others')
    pie_values.append(other_pct)

ax2.pie(pie_values, labels=pie_labels, autopct='%1.1f%%', startangle=90)
ax2.set_title(f'Top {top_n} Classes by Pixel Coverage')

plt.tight_layout()
plt.show()

print(f"\nTotal pixels analyzed: {total_pixels:,}")
print("\nTop 5 classes by pixel coverage:")
for i, (cls, pct) in enumerate(sorted_classes[:5], 1):
    print(f"{i}. {VOC_CLASSES[cls]}: {pct:.2f}% ({pixel_counts[cls]:,} pixels)")


In [ ]:
## 7. Object Co-occurrence Analysis

def analyze_class_cooccurrence(image_ids, max_samples=300):
    """Analyze which classes tend to appear together in images"""
    cooccurrence_matrix = np.zeros((len(VOC_CLASSES), len(VOC_CLASSES)), dtype=int)
    
    for img_id in image_ids[:max_samples]:
        mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
        if os.path.exists(mask_path):
            mask = np.array(Image.open(mask_path))
            unique_classes = np.unique(mask)
            unique_classes = unique_classes[unique_classes != 255]
            
            # Update co-occurrence matrix
            for i, cls1 in enumerate(unique_classes):
                for cls2 in unique_classes:
                    if cls1 < len(VOC_CLASSES) and cls2 < len(VOC_CLASSES):
                        cooccurrence_matrix[cls1, cls2] += 1
    
    return cooccurrence_matrix

print("Analyzing class co-occurrence...")
cooccurrence = analyze_class_cooccurrence(train_ids, max_samples=300)

# Normalize by diagonal (how often each class appears)
diagonal = np.diag(cooccurrence).copy()
normalized_cooccurrence = np.zeros_like(cooccurrence, dtype=float)

for i in range(len(VOC_CLASSES)):
    for j in range(len(VOC_CLASSES)):
        if diagonal[i] > 0:
            normalized_cooccurrence[i, j] = cooccurrence[i, j] / diagonal[i]

# Visualize
plt.figure(figsize=(14, 12))
sns.heatmap(normalized_cooccurrence, 
            xticklabels=VOC_CLASSES, 
            yticklabels=VOC_CLASSES,
            cmap='YlOrRd', 
            annot=False, 
            fmt='.2f',
            cbar_kws={'label': 'Co-occurrence Probability'})
plt.title('Class Co-occurrence Matrix\n(Probability that column class appears when row class is present)')
plt.xlabel('Class')
plt.ylabel('Class')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print interesting co-occurrences (excluding self)
print("\nTop 10 Class Pairs that Often Appear Together:")
pairs = []
for i in range(len(VOC_CLASSES)):
    for j in range(i+1, len(VOC_CLASSES)):
        if diagonal[i] > 0 and diagonal[j] > 0:
            avg_prob = (cooccurrence[i, j] / diagonal[i] + cooccurrence[j, i] / diagonal[j]) / 2
            pairs.append((VOC_CLASSES[i], VOC_CLASSES[j], avg_prob, cooccurrence[i, j]))

pairs.sort(key=lambda x: x[2], reverse=True)
for i, (cls1, cls2, prob, count) in enumerate(pairs[:10], 1):
    print(f"{i}. {cls1} ↔ {cls2}: {prob:.2%} co-occurrence ({count} times)")


In [ ]:
## 8. Segmentation Complexity Analysis

def analyze_segmentation_complexity(image_ids, max_samples=300):
    """Analyze the complexity of segmentation masks"""
    num_classes_per_image = []
    num_objects_per_image = []
    
    for img_id in image_ids[:max_samples]:
        mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
        obj_mask_path = os.path.join(SEGMENTATION_OBJECT_DIR, f'{img_id}.png')
        
        if os.path.exists(mask_path):
            # Count unique classes
            mask = np.array(Image.open(mask_path))
            unique_classes = np.unique(mask)
            unique_classes = unique_classes[unique_classes != 255]
            num_classes_per_image.append(len(unique_classes))
            
            # Count unique objects
            if os.path.exists(obj_mask_path):
                obj_mask = np.array(Image.open(obj_mask_path))
                unique_objects = np.unique(obj_mask)
                unique_objects = unique_objects[unique_objects != 255]
                unique_objects = unique_objects[unique_objects != 0]  # Exclude background
                num_objects_per_image.append(len(unique_objects))
    
    return num_classes_per_image, num_objects_per_image

print("Analyzing segmentation complexity...")
num_classes_per_img, num_objects_per_img = analyze_segmentation_complexity(train_ids, max_samples=300)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution of number of classes per image
axes[0].hist(num_classes_per_img, bins=range(1, max(num_classes_per_img)+2), 
             edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Classes per Image')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Classes per Image')
axes[0].grid(axis='y', alpha=0.3)

# Distribution of number of objects per image
if num_objects_per_img:
    axes[1].hist(num_objects_per_img, bins=range(1, max(num_objects_per_img)+2), 
                 edgecolor='black', alpha=0.7, color='coral')
    axes[1].set_xlabel('Number of Objects per Image')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of Object Instances per Image')
    axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nClasses per Image:")
print(f"  Mean: {np.mean(num_classes_per_img):.2f}")
print(f"  Median: {np.median(num_classes_per_img):.2f}")
print(f"  Min: {min(num_classes_per_img)}, Max: {max(num_classes_per_img)}")

if num_objects_per_img:
    print(f"\nObjects per Image:")
    print(f"  Mean: {np.mean(num_objects_per_img):.2f}")
    print(f"  Median: {np.median(num_objects_per_img):.2f}")
    print(f"  Min: {min(num_objects_per_img)}, Max: {max(num_objects_per_img)}")


In [ ]:
## 9. Object Size Analysis

def analyze_object_sizes(image_ids, max_samples=200):
    """Analyze the relative size of objects in images"""
    class_sizes = defaultdict(list)  # List of relative sizes (% of image) per class
    
    for img_id in image_ids[:max_samples]:
        mask_path = os.path.join(SEGMENTATION_CLASS_DIR, f'{img_id}.png')
        if os.path.exists(mask_path):
            mask = np.array(Image.open(mask_path))
            total_pixels = mask.size
            
            unique_classes = np.unique(mask)
            unique_classes = unique_classes[unique_classes != 255]
            unique_classes = unique_classes[unique_classes != 0]  # Exclude background
            
            for cls in unique_classes:
                cls_pixels = np.sum(mask == cls)
                relative_size = (cls_pixels / total_pixels) * 100
                class_sizes[cls].append(relative_size)
    
    return class_sizes

print("Analyzing object sizes...")
class_sizes = analyze_object_sizes(train_ids, max_samples=250)

# Calculate statistics
size_stats = {}
for cls, sizes in class_sizes.items():
    if cls < len(VOC_CLASSES):
        size_stats[VOC_CLASSES[cls]] = {
            'mean': np.mean(sizes),
            'median': np.median(sizes),
            'min': np.min(sizes),
            'max': np.max(sizes),
            'count': len(sizes)
        }

# Sort by mean size
sorted_stats = sorted(size_stats.items(), key=lambda x: x[1]['mean'], reverse=True)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Box plot for top classes
top_classes = [item[0] for item in sorted_stats[:12]]  # Top 12 by mean size
box_data = [class_sizes[VOC_CLASSES.index(cls)] for cls in top_classes]

bp = ax1.boxplot(box_data, labels=top_classes, patch_artist=True)
for patch, cls_name in zip(bp['boxes'], top_classes):
    cls_idx = VOC_CLASSES.index(cls_name)
    if cls_idx < len(VOC_COLORMAP):
        patch.set_facecolor([c/255.0 for c in VOC_COLORMAP[cls_idx]])
        patch.set_alpha(0.7)

ax1.set_ylabel('Relative Size (% of Image)')
ax1.set_xlabel('Class')
ax1.set_title('Distribution of Object Sizes (Top 12 Classes)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Mean size comparison
classes_for_bar = [item[0] for item in sorted_stats[:15]]
mean_sizes = [item[1]['mean'] for item in sorted_stats[:15]]

bars = ax2.barh(range(len(classes_for_bar)), mean_sizes, edgecolor='black', alpha=0.7)
ax2.set_yticks(range(len(classes_for_bar)))
ax2.set_yticklabels(classes_for_bar)
ax2.set_xlabel('Mean Relative Size (% of Image)')
ax2.set_title('Average Object Size by Class (Top 15)')
ax2.invert_yaxis()

# Color bars
for i, cls_name in enumerate(classes_for_bar):
    cls_idx = VOC_CLASSES.index(cls_name)
    if cls_idx < len(VOC_COLORMAP):
        bars[i].set_color([c/255.0 for c in VOC_COLORMAP[cls_idx]])

plt.tight_layout()
plt.show()

print("\nObject Size Statistics (% of image):")
print("-" * 70)
for cls_name, stats in sorted_stats[:10]:
    print(f"{cls_name:15s} - Mean: {stats['mean']:5.2f}%, Median: {stats['median']:5.2f}%, "
          f"Range: [{stats['min']:5.2f}%, {stats['max']:5.2f}%]")


In [ ]:
## 10. Dataset Summary

print("=" * 80)
print("PASCAL VOC 2012 SEMANTIC SEGMENTATION DATASET - SUMMARY")
print("=" * 80)

print("\n📊 DATASET SIZE:")
print(f"  • Training images: {len(train_ids)}")
print(f"  • Validation images: {len(val_ids)}")
print(f"  • Total images: {len(trainval_ids)}")

print("\n🎨 CLASSES:")
print(f"  • Total number of classes: {len(VOC_CLASSES)} (20 object classes + background)")
print(f"  • Classes: {', '.join(VOC_CLASSES[1:])}")

if dimensions:
    print("\n📐 IMAGE DIMENSIONS:")
    print(f"  • Width range: {min(widths)} - {max(widths)} pixels (mean: {np.mean(widths):.0f})")
    print(f"  • Height range: {min(heights)} - {max(heights)} pixels (mean: {np.mean(heights):.0f})")
    print(f"  • Aspect ratio range: {min(aspect_ratios):.2f} - {max(aspect_ratios):.2f}")

if class_counts:
    top_3_classes = class_counts.most_common(3)
    print("\n🏆 MOST FREQUENT CLASSES (in sampled images):")
    for i, (cls, count) in enumerate(top_3_classes, 1):
        print(f"  {i}. {VOC_CLASSES[cls]}: appears in {count} images")

if sorted_classes:
    print("\n🎯 PIXEL COVERAGE (from sampled images):")
    for i, (cls, pct) in enumerate(sorted_classes[:3], 1):
        print(f"  {i}. {VOC_CLASSES[cls]}: {pct:.2f}% of all pixels")

if num_classes_per_img:
    print("\n🧩 SEGMENTATION COMPLEXITY:")
    print(f"  • Average classes per image: {np.mean(num_classes_per_img):.2f}")
    print(f"  • Range: {min(num_classes_per_img)} - {max(num_classes_per_img)} classes per image")

print("\n💡 KEY INSIGHTS:")
print("  • The dataset exhibits class imbalance (background dominates)")
print("  • Images contain multiple objects with varying sizes")
print("  • Common co-occurrences reflect real-world scenarios (e.g., person+chair)")
print("  • Variable image dimensions require careful preprocessing")

print("\n" + "=" * 80)
print("EDA Complete! The dataset is ready for semantic segmentation tasks.")
print("=" * 80)
